**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Dec 18 09:47:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'bandicam 2020-10-04 12-19-20-585.mp4'	'ML Questions.pdf'
 Classroom				 mml-book.pdf
'Colab Notebooks'			'photoshop softwares'
 faces					 RSVP.gform
 images


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 14645 (delta 8), reused 11 (delta 4), pack-reused 14621
Receiving objects: 100% (14645/14645), 13.25 MiB | 25.51 MiB/s, done.
Resolving deltas: 100% (9963/9963), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [6]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [10]:
!echo "screw" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [11]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-12-18 09:51:01--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   139KB/s    in 15m 39s 

2020-12-18 10:06:40 (169 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [21]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/2020-11-11_14-03-17_551.heic  
  inflating: data/obj/2020-11-11_14-03-17_551.txt  
  inflating: data/obj/2020-11-11_14-03-22_877.heic  
  inflating: data/obj/2020-11-11_14-03-22_877.txt  
  inflating: data/obj/2020-11-11_14-03-32_179.heic  
  inflating: data/obj/2020-11-11_14-03-32_179.txt  
  inflating: data/obj/2020-11-11_14-03-33_657.heic  
  inflating: data/obj/2020-11-11_14-03-33_657.txt  
  inflating: data/obj/2020-11-11_14-06-16_168.heic  
  inflating: data/obj/2020-11-11_14-06-19_528.heic  
  inflating: data/obj/2020-11-11_14-06-19_528.txt  
  inflating: data/obj/2020-11-11_14-06-52_600.heic  
  inflating: data/obj/2020-11-11_14-06-52_600.txt  
  inflating: data/obj/2020-11-11_14-06-59_907.heic  
  inflating: data/obj/2020-11-11_14-07-09_266.heic  
  inflating: data/obj/2020-11-11_14-07-09_266.txt  
  inflating: data/obj/2020-11-11_14-07-17_218.heic  
  inflating: data/obj/2020-11-11_14-07-18_480.heic  
  inflating: dat

In [22]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.621032 0.614749 0.107639 0.107143

['15', '0.621032', '0.614749', '0.107639', '0.107143']
0 data/obj/2020-11-11_14-09-37_823.txt
0 0.621032 0.614749 0.107639 0.107143
15 0.768725 0.619544 0.091022 0.084987

['15', '0.768725', '0.619544', '0.091022', '0.084987']
0 data/obj/2020-11-11_14-09-37_823.txt
0 0.768725 0.619544 0.091022 0.084987
15 0.454530 0.580605 0.194114 0.139385

['15', '0.454530', '0.580605', '0.194114', '0.139385']
1 data/obj/2020-11-11_14-10-11_309.txt
0 0.454530 0.580605 0.194114 0.139385
15 0.500000 0.603795 0.367725 0.291419

['15', '0.500000', '0.603795', '0.367725', '0.291419']
2 data/obj/2020-11-11_14-07-52_739.txt
0 0.500000 0.603795 0.367725 0.291419
15 0.628307 0.545759 0.429894 0.252728

['15', '0.628307', '0.545759', '0.429894', '0.252728']
3 data/obj/2020-11-11_14-08-10_778.txt
0 0.628307 0.545759 0.429894 0.252728
15 0.354663 0.542163 0.160714 0.154431

['15', '0.354663', '0.542163', '0.160714', '0.154431']
4 data/obj/2020-11-11_14-09-46_146 (1).txt
0 

In [32]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

[]


In [33]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [25]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x 